# DNN: backprop

Deep learning applications have been tremendously facilitated by various open-source libraries such as [TensorFlow](https://www.tensorflow.org/), [PyTorch](https://pytorch.org/) and [Apache MXNet](https://mxnet.apache.org/). This notebook aims at delivering a deeper understanding of what happens inside the black box by 

* demonstrating Gradient Descent in 1D;
* demonstrating different levels of implementing the training loop with TensorFlow;
* creating a fully-connected DNN from scratch based on `numpy`.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# helpers
from tqdm import trange
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')

---

# Gradient Descent in 1D

Gradient Descent is a first-order iterative approach to solve a non-linear programming problem with continuous differentiability. To better understand it, let us solve a 1D problem:

> $\underset{x}{\mathrm{argmin}}\ f(x)$

The alogrithm is

> $x_{k+1} = x_{k}-\eta \left . \dfrac{\partial f}{\partial x} \right\vert _{x=x_k},\quad \eta\in R^+$

The algorithm can be simply implemented as

In [ ]:
def GD_1D(f, df, x0, eta, n_iter, alpha=0.):
    """
    GD in 1D
    :param f: target function
    :param df: derivative of target function
    :param x0: inital guess
    :param eta: learning rate
    :param n_iter: number of interations
    :param alpha: momentum decay factor  (default=0., no momentum)
    :return: trajactory of convergence
    """
    # allocate arrays
    x = np.zeros((n_iter + 1))
    y = np.zeros((n_iter + 1))
    
    # initial guess
    x[0] = x0
    y[0] = f(x0)
    
    # iteration
    dx = 0.
    for k in range(n_iter):
        dx = - eta * df(x[k]) + alpha * dx
        x[k + 1] = x[k] + dx
        y[k + 1] = f(x[k + 1])
        
    # return trajactory
    return x, y

Now, we can choose a non-linear function to try out GD:

In [ ]:
# f(x)
def f(x):
    return 3 * x ** 4 - 6 * x ** 2 - 2 * x + 2

# ∂f/∂x
def df(x):
    return 12 * x ** 3 - 12 * x - 2

# ∂f/∂x by AutoDiff
def df_auto(x):
    # tell TensorFlow that x is a variable 
    x_ = tf.Variable(x)
    # build graph with GradientTape
    with tf.GradientTape() as tape:
        y = f(x_)
    # compute gradient
    dy_dx = tape.gradient(y, x_)
    return dy_dx.numpy()
    
# initial guess
x0 = 0

# learning rate
eta = 0.01

# num iter
n_iter = 10

# perform GD
x, y = GD_1D(f, df_auto, x0, eta, n_iter, alpha=0.)

# display results
print(f'final x: {x[-1]}')
print(f'final y: {y[-1]}')

# plot range
x_range = np.linspace(-1.5, 1.5, 1000)
plt.figure(dpi=200)
plt.plot(x_range, f(x_range), ls='--', label='$f(x)$')
plt.plot(x, y, label='Trajectory', marker='.')
plt.scatter(x[0], y[0], c='b', zorder=100, s=100, marker='+')
plt.scatter(x[-1], y[-1], c='r', zorder=100, s=100, marker='+')
plt.legend()
plt.show()

## Exercises

Implement [AdaGrad](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad), [RMSProp](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) and [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam) for 1D Gradient Descent.  

---

# Training Loop in TensorFlow

In [DNN_basics.ipynb](DNN_basics.ipynb) and [DNN_practical.ipynb](DNN_practical.ipynb), we have conducted training by simply calling `tf.keras.Model.fit()`. It is the most convenient way for simple network architectures and loss functions. When a more complicated architecture or loss function is required, we may have to use some lower-level APIs rather than a simple call of `model.fit()`. Here we introduce the two other levels: 

* overloaded member function `train_step()`, which exposes backpropagation to users, and
* handcoded training loop, which exposes epoch and mini-batch loops to users.


### Load dataset 

It is the same as in [DNN_basics.ipynb](DNN_basics.ipynb):

In [ ]:
# load dataset
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# normalise images
train_images = train_images / 255.0
test_images = test_images / 255.0

### Method 1: `model.fit()`

The simplest way to train a DNN is to create a build-in `Sequential` model and call `model.fit()`, as what we did in [DNN_basics.ipynb](DNN_basics.ipynb):

In [ ]:
# build the network architecture
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(10, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# train
history = model.fit(train_images, train_labels, epochs=2, batch_size=32)

### Method 2: `train_step()`

Still calling `model.fit()`, we can customize what happens in `model.fit()` by overloading the member function `train_step()` of the class `tf.keras.Model`. This function contains the process of forward and back propagation, useful for more complex architectures and loss functions. In [VAE_basics.ipynb](../VAE/VAE_basics.ipynb), for example, we use this method to implement the KL-divergence loss in a variational autoencoder.

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
# build the network architecture
inputs = keras.Input(shape=(28, 28))
flattened = Flatten(input_shape=(28, 28))(inputs)
outputs = Dense(10)(flattened)
model = CustomModel(inputs, outputs)

# compile the model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# train
history = model.fit(train_images, train_labels, epochs=2, batch_size=32)

### Method 3: Handcoded training loop

The `train_step()` method can become insufficiently flexible for even more complicated architectures or loss functions. For example, `train_step()` only allows for one argument, but we may want more arguments to control the workflow. One can reach the maximum flexibility by coding training loop explicitly. In `GAN_basics.ipynb`, for example, we use this method to implement a Generative Adversarial Network or GAN.

First, we create a model, not necessarily a `Sequential` one:

In [ ]:
# build the network architecture
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(10, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Next, we write our own `train_step()` alternative, named `my_own_train_step()`, which is a generic Python function with any number of arguments (here we pass images and labels) and returns (here we return the loss):

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def my_own_train_step(images, labels):
    # Unpack the data. Its structure depends on your model and
    # on what you pass to `fit()`.
    x, y = images, labels

    with tf.GradientTape() as tape:
        y_pred = model(x)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        loss = model.compiled_loss(y, y_pred, regularization_losses=model.losses)

    # Compute gradients
    trainable_vars = model.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    # Update weights
    model.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # Update metrics (includes the metric that tracks the loss)
    model.compiled_metrics.update_state(y, y_pred)
    
    # return loss
    return loss

Finally, the training loop can be implemented as follows:

In [ ]:
# epochs
EPOCHS = 2
BATCH_SIZE = 32
NUM_IMAGES = train_images.shape[0]

# epoch loop
tstart = time.time()
for epoch in range(EPOCHS):
    # mini-batch loop
    epochs = trange(0, NUM_IMAGES, BATCH_SIZE)
    for begin_index in epochs:
        end_index = min(begin_index + BATCH_SIZE, NUM_IMAGES)
        images = train_images[begin_index:end_index]
        labels = train_labels[begin_index:end_index]
        # call my_own_train_step
        loss = my_own_train_step(images, labels)
        epochs.set_description(f"Epoch %d, Loss=%.2f" % (epoch, loss))

## Exercises

In the above implementation, we manually calculated the starting and ending indices of the mini-batch. Use [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) to automate this process -- and it can do much more than just this.

---

# Gradient Descent and Backprop for DNN

Before the implementation, we must first understand how a neural network works. In a nutshell, a neural network trys to minimise the prediction error or the loss, $\epsilon$, by tuning the model parameters $\mathbf{w}$. **Gradient Descent** is the most fundamental algorithm for this purpose, which iteratively updates $\mathbf{w}$ in the "gradient" direction $\nabla\epsilon=\dfrac{\partial \epsilon}{\partial \mathbf{w}}$. This can be achieved through cycles of forward and backward propagations.

### Forward propagation

For a fully-connected dense layer, the trainable model parameters include the **weights** and the **biases**. Let $N_l$ denote the number of neurons in the $l$-th layer. The $l$-th layer then contains $N_{l}\times N_{l-1}$ weights and $N_l$ biases, as denoted respectively by $w^l_{ij}$ and $b^l_i$, where $1\le i \le N_{l}$ and $1\le j \le N_{l-1}$, as shown in the following figure.

<img src="https://github.com/stfc-sciml/sciml-workshop-v3/blob/master/course_3.0_with_solutions/markdown_pic/fwdd.png?raw=1" width="40%">

The forward propagation passes the input data to the first hidden layer and computes the values on its neurons (using the current weights and biases), which are then "**activated**" and passed to the second layer and so on until the output layer is reached. The pre-activation value at the $i$-th neuron in the $l$-th layer, as denoted by $z^l_{i}$, is computed by

$$z^l_{i}=\sum_{j=1}^{N_{l-1}} w^l_{ij} a^{l-1}_{j} + b^l_i,\quad i=1,2,\cdots,N_{l},
\label{eq:zi} \tag{1}
$$

where $a^{l-1}_{j}$ is the post-activation value at the $j$-th neuron in the $(l-1)$-th layer. Then the post-activation values of the $l$-th layer are computed using the given activation function $f_l$, such as `ReLU` and `sigmoid`:

$$a^l_{i}=f_l\left(z^l_{i}\right),\quad i=1,2,\cdots,N_{l}.
\label{eq:ai} \tag{2}
$$

Next, $a^{l}_{i}$ will be passed to the $(l+1)$-th layer to compute $z^{l+1}_{k}$, $k=1,2,\cdots,N_{l+1}$.

#### Implementation in Python

Assume that we have a class called `Layer` that has properties `w` and `b` and a member function `activation_func`. The forward propagation through this layer can be implemented as follows, passing `a` from the previous layer:

```python
def forward(self, a_prev):
    # pre-activation
    self.z = np.dot(self.w, a_prev) + self.b
    # post-activation
    self.a = self.activation_func(self.z)
    # return a to feed the next layer
    return self.a
```

Note that we store `z` and `a` (by using `self.`) because they will be needed for backpropagation.


### Backward propagation of errors

Suppose that our DNN has $M$ layers in total, so the $M$-th layer will be the output layer and $a^{M}_{k}$ the final prediction. The vector-valued error is computed by subtracting the prediction $a^{M}_{k}$ and the ground truth $y_{k}$. Let us use the Mean Squared Error (MSE) as the scalar-valued loss function, i.e.,

$$\epsilon=\dfrac{1}{N_M}\sum_{k=1}^{N_M}(a^{M}_{k}-y_{k})^2.
\label{eq:eps}\tag{3}
$$


The purpose of backpropagation is to find the model gradients $\dfrac{\partial \epsilon}{\partial w^l_{ij}}$ and $\dfrac{\partial \epsilon}{\partial b^l_{i}}$. They can be evaluated based on the **chain rule**:


$$\dfrac{\partial \epsilon}{\partial w^l_{ij}}
=\dfrac{\partial \epsilon}{\partial z^l_{i}}\dfrac{\partial z^l_{i}}{\partial w^l_{ij}}
=\dfrac{\partial \epsilon}{\partial a^l_{i}}\dfrac{d a^l_{i}}{d z^l_{i}}\dfrac{\partial z^l_{i}}{\partial w^l_{ij}}.
\label{eq:w}\tag{4}
$$

The second term on the R.H.S. of eq. $\eqref{eq:w}$, $\dfrac{d a^l_{i}}{d z^l_{i}}$, is the derivative of the activation funciton, $f_l'\left({z^l_{i}}\right)$, and the third term $\dfrac{\partial z^l_{i}}{\partial w^l_{ij}}$ simply $a^{l-1}_{j}$. The first term is more complicated because $a^l_{i}$ contributes to $\epsilon$ via the whole $(l+1)$-th layer, that is,
$\dfrac{\partial \epsilon}{\partial a^l_{i}}= \sum\limits_{k=1}^{N_{l+1}} 
\dfrac{\partial \epsilon}{\partial z^{l+1}_{k}}\dfrac{\partial z^{l+1}_{k}}{\partial a^l_{i}}  =
\sum\limits_{k=1}^{N_{l+1}} 
\dfrac{\partial \epsilon}{\partial z^{l+1}_{k}}w_{ki}^{l+1}.
$
Inserted with all these three terms, the above gradient can be eventually rearrange as (also considering the output layer, $l=M$):

$$\dfrac{\partial \epsilon}{\partial w^l_{ij}}
=\dfrac{\partial \epsilon}{\partial z^l_{i}}a^{l-1}_{j}
=\dfrac{\partial \epsilon}{\partial a^l_{i}}
f'_l\left({z^l_{i}}\right)a^{l-1}_{j}, \quad \dfrac{\partial \epsilon}{\partial a^l_{i}}=
\begin{cases}
\dfrac{2}{N_M}(a^{l}_{i}-y_{i}),&l=M;\\
\sum\limits_{k=1}^{N_{l+1}} w_{ki}^{l+1}\dfrac{\partial \epsilon}{\partial z^{l+1}_{k}},&l<M.
\end{cases}
\label{eq:wij}\tag{5}
$$


Obviously, the above gradient must be computed in a backward sequence (from $l=M$ to $l=1$), with the "errors" $\dfrac{\partial \epsilon}{\partial z^{l+1}_{k}}$ passed from the $(l+1)$-th layer to the $l$-th layer; this is why the process is called backpropagtion. For the biases, it is straightforward to show that 
$\dfrac{\partial \epsilon}{\partial b^l_{i}}=\dfrac{\partial \epsilon}{\partial z^l_{i}}$ because 
$\dfrac{\partial z^l_{i}}{\partial b^l_{i}}=1$.


#### Implementation in Python

Based on eq. $\eqref{eq:wij}$, the backpropagation can be coded as follows:

```python
# input: 1) ∂ε/∂a of this layer but computed in the next layer
#        2) a of the previous layer
def backward(self, de_da, a_prev):
    # ∂ε/∂z (which is also ∂ε/∂b)
    de_dz = de_da * self.activation_func(self.z, derivative=True)
    # accumulate ∂ε/∂w, ∂ε/∂b
    self.de_dw += np.outer(de_dz, a_prev)
    self.de_db += de_dz
    # ∂ε/∂a to be passed to the previous layer
    de_da_prev = np.dot(self.w.T, de_dz)
    return de_da_prev
```

Here we accumulate the gradients (instead of directly updating the parameters) because we will employ Mini-batch Gradient Descent for parameter update, as introduced below.

###  Mini-batch Gradient Descent

Through one iteration of forward and backward propagations, each data in the dataset will yield a gradient. Then the question is which gradient we should use to update the model parameters. To answer this question, we will use an algorithm called **Mini-batch Gradient Descent**:

$$
\Delta w_{ij}=-\eta \dfrac{1}{B} \sum_{m=1}^{B}  \dfrac{\partial\epsilon_m}{\partial w_{ij}},
\label{eq:mini}\tag{6}
$$

where $B$ is a given **batch size**, $\dfrac{\partial\epsilon_m}{\partial w_{ij}}$ the gradient computed with the $m$-th data in the mini-batch and $\eta$ the **learning rate**. 


#### Batch size $B$

When $B$ is selected to be the total number of data in the dataset, the algorithm is usually referred to as **Batch Gradient Descent**. For a non-convex optimisation problem (which is generally the case in deep learning), Batch Gradient Descent can easily be trapped by local minima.

To help the algorithm to escape from local minima, we can add a bit noise to the trajectory of gradient descent by using a gradient averaged over a *random subset* of the dataset -- the so called **Mini-batch Stochastic Gradient Descent** or **Mini-batch Gradient Descent**. The noise level decreases with $B$. When $B=1$, the algorithm is commonly known as **Stochastic Gradient Descent**, which introduces the highest noise level and thus may suffer from slow convergence.  


#### Learning rate $\eta$
Now we have found the direction to update $w^l_{ij}$ and $b^l_{i}$, but we still need to determine the magnitude of the update. This will introduce another network parameter called the **learning rate**. In our implementation, we will use a constant learning rate. In applications, it is usually more efficient to use an adaptive learning rate, such as by using the Adam optimiser. 

> "The learning rate is perhaps the most important hyperparameter. If you have time to tune only one hyperparameter, tune the learning rate." -- Goodfellow, Deep Learning



---

# Implementation from Scratch

### 1. Activation functions

We first define some activation functions. They can be passed as an argument to create a `Layer` object.

In [ ]:
# relu
def relu(x, derivative=False):
    if derivative:
        return 1. * (x > 0)
    else:
        return x * (x > 0.)

# sigmoid
def sigmoid(x, derivative=False):
    if derivative:
        s = sigmoid(x)
        return s * (1. - s)
    else:
        return 1. / (1. + np.exp(-x))
    
# linear
def linear(x, derivative=False):
    if derivative:
        return np.full(x.shape, 1.)
    else:
        return x.copy()

### 2. Class `Layer`
The `Layer` class is where the model parameters and neuron values are stored and the layer-wise operations happen.

In [ ]:
# layer class
class Layer:
    # constructor
    def __init__(self, size, activation=linear, name='Untitled_Layer'):
        # we cannot allocate data here because 
        # the size of the previous layer is unknown
        self.size = size
        self.activation_func = activation
        self.name = name
        
    # intialise parameters
    def initParameters(self, prev_size):
        # weights and biases,
        # intialised with random numbers from the “standard normal” distribution
        self.w = np.random.randn(self.size, prev_size) * np.sqrt(1. / self.size)
        self.b = np.zeros((self.size,))
        
        # neuron values
        self.a = np.zeros((self.size,))
        self.z = np.zeros((self.size,))
        
        # accumulated gradients
        self.de_dw = np.zeros((self.size, prev_size))
        self.de_db = np.zeros((self.size,))
        
    # forward propagation
    def forward(self, a_prev):
        # pre-activation
        self.z = np.dot(self.w, a_prev) + self.b
        # post-activation
        self.a = self.activation_func(self.z)
        # return a to feed the next layer
        return self.a
    
    # backward propagation
    def backward(self, de_da, a_prev):
        # ∂ε/∂z (which is also ∂ε/∂b)
        de_dz = de_da * self.activation_func(self.z, derivative=True)
        # accumulate ∂ε/∂w, ∂ε/∂b
        self.de_dw += np.outer(de_dz, a_prev)
        self.de_db += de_dz
        # ∂ε/∂a to be passed to the previous layer
        de_da_prev = np.dot(self.w.T, de_dz)
        return de_da_prev
    
    # update parameters
    def updateParameters(self, learning_rate, batch_size):
        # update
        self.w -= (learning_rate / batch_size) * self.de_dw
        self.b -= (learning_rate / batch_size) * self.de_db
        
        # reset accumulated for the next mini-batch
        self.de_dw.fill(0.)
        self.de_db.fill(0.)
        
    # print
    def __str__(self):
        s = '%s\n' % self.name
        s += 'Size = %d\n' % self.size
        s += 'Activation = %s\n' % (self.activation_func.__name__,)
        try:
            s += 'Number of weights = %d\n' % self.w.size
            s += 'Number of biases = %d\n' % self.b.size
        except:
            s += 'Not set in a model.\n'
        return s

### 3. Class `Model`

The `Model` class contains many `Layer` objects and controls the workflow of training.

In [ ]:
# model class
class Model:
    # constructor
    def __init__(self, input_size, layers, name='Untitled_Model'):
        self.input_size = input_size
        self.layers = layers
        self.name = name
        
        # initialise layers
        prev_size = input_size
        for layer in self.layers:
            layer.initParameters(prev_size)
            prev_size = layer.size
            
    # forward propagation
    def forward(self, input_data):
        a_prev = input_data.copy()
        for layer in self.layers:
            a_prev = layer.forward(a_prev)
        return a_prev
    
    # backward propagation
    def backward(self, input_data, error):
        de_da = error.copy()
        # from the last to the second layers
        for ilayer in np.arange(len(self.layers) - 1, 0, -1):
            de_da = self.layers[ilayer].backward(de_da, self.layers[ilayer - 1].a)
        # the first layer
        self.layers[0].backward(de_da, input_data)
        
    # update parameters
    def updateParameters(self, learning_rate, batch_size):
        for layer in self.layers:
            layer.updateParameters(learning_rate, batch_size)
    
    # compute loss and accuracy for history
    def computeLossAccHistory(self, xs, ys, history, key_prefix, iep):
        for x, y in zip(xs, ys):
            pred_y = self.forward(x)
            history[key_prefix + 'loss'][iep] += np.linalg.norm(pred_y - y) / len(pred_y)
            history[key_prefix + 'acc'][iep] += int(np.argmax(pred_y) == np.argmax(y))
        history[key_prefix + 'loss'][iep] /= xs.shape[0]
        history[key_prefix + 'acc'][iep] /= xs.shape[0]
            
    # train
    def train(self, train_x, train_y, 
              epochs=2, batch_size=32, learning_rate=0.1, validation_data=None, verbose=1):
        # number of data
        ndata = train_x.shape[0]
        
        # number of data in mini-batches
        n_mini_batch = ndata // batch_size + int(ndata % batch_size > 0)
        n_data_mini_batches = np.full((n_mini_batch,), batch_size)
        # the last one may have fewer
        n_data_mini_batches[-1] = ndata - (n_mini_batch - 1) * batch_size
        
        # history
        history = {'loss': np.zeros((epochs,)), 'acc': np.zeros((epochs,))}
        if validation_data is not None:
            history['val_loss'] = np.zeros((epochs,))
            history['val_acc'] = np.zeros((epochs,))
        
        # epoch loop
        start_time = time.time()
        for iep in np.arange(epochs):
            # data must be shuffled before each epoch
            permute = np.random.permutation(ndata)
            train_x_sh = train_x[permute].copy()
            train_y_sh = train_y[permute].copy()
            
            # mini-batch loop
            for ibatch in np.arange(n_mini_batch):
                # data loop
                for idata in np.arange(ibatch * batch_size, n_data_mini_batches[ibatch]):
                    # forward
                    pred_y = self.forward(train_x_sh[idata])
                    # compute MSE
                    error = (pred_y - train_y_sh[idata]) * 2. / len(pred_y)
                    # backward
                    self.backward(train_x_sh[idata], error)
                # update parameters
                self.updateParameters(learning_rate, batch_size)
            
            # history on training data
            self.computeLossAccHistory(train_x, train_y, history, '', iep)
            
            # history on validation data
            if validation_data is not None:
                self.computeLossAccHistory(validation_data[0], validation_data[1], history, 'val_', iep)
                
            # print training info
            if verbose > 0 and (iep % verbose == 0 or iep == epochs - 1):
                print('Epoch %d: ' % iep, end='')
                print('loss = %f; acc = %f' % (history['loss'][iep], history['acc'][iep]), end='')
                if validation_data is not None:
                    print('; val_loss = %f; val_acc = %f; ' % (history['val_loss'][iep], 
                                                             history['val_acc'][iep]), end='')
                print('elapsed time = %f' % (time.time() - start_time,))
        if (verbose > 0):
            print('Finished %d epochs, elapsed time = %f' % (epochs, time.time() - start_time))
            
        # return history
        return history
    
    # predict
    def predict(self, pred_x):
        pred_y = []
        for x in zip(pred_x):
            pred_y.append(self.forward(x[0]))
        return np.array(pred_y) 
    
    # print
    def __str__(self):
        s = 'Model name: %s\n' % self.name
        s += 'Input size = %d\n' % (self.input_size,)
        s += 'Number of layers = %d\n' % (len(self.layers),)
        s += '========================================\n'
        for ilayer, layer in enumerate(self.layers):
            s += 'Layer %d: ' % ilayer
            s += str(layer)
            s += '----------------------------------------\n'
        return s

---

## Application

Now we can use our own DNN to classify the `fashion-mnist` dataset as we did in [DNN_basics.ipynb](DNN_basics.ipynb) based on Keras.

In [ ]:
# tensorflow (only for the dataset)
from tensorflow import keras

### 0. Load  the dataset

Our implementation does not do input flattening and output one-hot encoding internally, so we have to do them manually after loading the data. Also, because our implementation is unoptimised, we will only use 20% of the dataset. 

In [ ]:
# load dataset
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# normalise images
train_images = train_images / 255.0
test_images = test_images / 255.0

# flatten images by numpy.ndarray.reshape()
npix = train_images.shape[1]
input_size = npix * npix
train_images = train_images.reshape((train_images.shape[0], input_size))
test_images = test_images.reshape((test_images.shape[0], input_size))

# one-hot encoding for labels by numpy.eye()
train_labels = np.eye(10)[train_labels]
test_labels = np.eye(10)[test_labels]

# use 20% of the dataset
train_images = train_images[0:12000]
train_labels = train_labels[0:12000]
test_images = test_images[0:2000]
test_labels = test_labels[0:2000]

### 1. Build the model

In [ ]:
# layers
hidden_layer = Layer(128, activation=relu, name='Hidden')
output_layer = Layer(10, activation=sigmoid, name='Output')

# model
model = Model(input_size, [hidden_layer, output_layer], name="DNN for fashion-mnist")

# print summary
print(model)

### 2. Train the model

Here we need a large number of epochs (1000) to obtain an accuracy comparable to what we had done in [DNN_basics.ipynb](DNN_basics.ipynb) with 50 epochs. Given that we only use 20% of the dataset, our Mini-batch Gradient Descent algorithm is still three times less efficient in convergence than the Adam optimiser implemented in Keras. Use a smaller `epochs` (100~200) for a faster but less accurate result.

In [ ]:
# train the model
history = model.train(train_images, train_labels, 
                      epochs=200, batch_size=32, learning_rate=.3,
                      validation_data=(test_images, test_labels))

In [ ]:
# plot accuracy
plt.figure(dpi=100, figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['acc'], label='Accuracy on training data')
plt.plot(history['val_acc'], label='Accuracy on test data')
plt.legend()
plt.title("Accuracy")

# plot loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Loss on training data')
plt.plot(history['val_loss'], label='Loss on test data')
plt.legend()
plt.title("Loss")
plt.show()

### 3. Make predictions

After training, we can use `model.predict` to make predictions:

In [ ]:
# make predictions
pred_labels = model.predict(test_images)
print('Number of test images: %d' % test_images.shape[0])
print('Number of correct predictions: %d' % 
      np.where(np.argmax(pred_labels, axis=1) == np.argmax(test_labels, axis=1))[0].size)

## Exercises

* **Easy**: Add an option to `Model` class to allow for a different method to initialise weights and biases, such as Glorot uniform [original paper](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf). 

* **Medium**: Add the dropout rate as a property of `Layer`. Randomly select neurons based on this rate and drop them out by zeroing their values in both forward and backward propagations. 

* **Hard**: Implement Implement [AdaGrad](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad), [RMSProp](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) or [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam).